In [19]:
import numpy as np 
import cv2
import shutil
import random
import os

In [20]:
def SplitDataBeforeAugmentation(source_folders, test_folder="", test_split=0.2):
    if not os.path.exists(test_folder):
        os.makedirs(test_folder)
    
    total_moved = 0
    
    for folder_path in source_folders:
        folder_name = os.path.basename(folder_path)
        
        image_files = [
            f for f in os.listdir(folder_path)
            if f.lower().endswith('.jpg')
        ]
        
        num_test = int(len(image_files) * test_split)
        test_images = random.sample(image_files, num_test)
        
        for img_name in test_images:
            file_path = os.path.join(folder_path, img_name)  # ✅ FIX
            
            img = cv2.imread(file_path)
            if img is None:
                os.remove(file_path)
                print(f"{file_path} is removed, corrupted image")
                continue
            
            dst = os.path.join(test_folder, img_name)
            shutil.move(file_path, dst)
        
        print(f"Moved {num_test} images from {folder_name} to test set")
        total_moved += num_test
    
    print(f"\nTotal test images: {total_moved}")
    return total_moved


In [21]:
def LoadImagesFromFolder(folder_path):
    """Load all images from a folder"""
    images = []
    image_files = [f for f in os.listdir(folder_path) 
                  if f.lower().endswith('.jpg')]
    
    for img_name in image_files:
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    
    return images

In [22]:
def reverseRow(img):
    return img[: ,::-1]

In [23]:
def reverseEntireMatrix(img):
    return img[::-1 , ::-1]

In [24]:
def RotateImage(img , angle = None):
    h, w = img.shape[:2]
    center = (w // 2, h // 2)
    if angle is None:
        angle = np.random.uniform(-36, 36) 
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    rotated = cv2.warpAffine(img, M, (w, h), borderMode=cv2.BORDER_REFLECT)
    return rotated

In [25]:
def ZoomImage(img , zoom_factor = None):
    h, w = img.shape[:2]
    if zoom_factor is None:
        zoom_factor = np.random.uniform(0.8, 0.95) 
    
    nh, nw = int(h * zoom_factor), int(w * zoom_factor)
    if nh >= h or nw >= w:
        return img  
    
    start_h = np.random.randint(0, h - nh + 1)
    start_w = np.random.randint(0, w - nw + 1)
    
    zoomed_img = img[start_h:start_h+nh, start_w:start_w+nw]

    zoomed_img = cv2.resize(zoomed_img, (w, h))
    return zoomed_img

In [26]:
def FlipImageHorizontally(img):
    img = reverseRow(img)
    return img

In [27]:
def FlipImageVertically(img):
    img =  reverseEntireMatrix(img)
    return img

In [28]:
def ChangeBrightness(img, value=None):
    if value is None:
        value = np.random.randint(-20, 20)
    img = img.astype(np.int16) + value
    img = np.clip(img, 0, 255)
    return img.astype(np.uint8)

In [29]:

def ChangeContrast(img, alpha=None):
    if alpha is None:
        alpha = np.random.uniform(0.9, 1.1)
    img = img.astype(np.float32) * alpha
    img = np.clip(img, 0, 255)
    Constrasted_image =  img.astype(np.uint8)
    return Constrasted_image

In [30]:
def ChangeSaturation(img):
    hsv_image = cv2.cvtColor(img , cv2.COLOR_BGR2HSV)
    factor = np.random.randint(0.7, 1.3)
    hsv_image[: , : , 1] = hsv_image[: , : , 1] * factor
    hsv_image[: , : , 1] = np.clip(hsv_image[: , : , 1] , 0 , 255)
    BGR_image = cv2.cvtColor(img , cv2.COLOR_HSV2BGR).astype(np.uint8)
    return BGR_image

In [31]:
def GaussianBlur(img):
    kernel_size = np.random.choice([3 , 5 ,7])
    kernel = (kernel_size , kernel_size)
    Blured_image = cv2.GaussianBlur(img , kernel , 0)
    return Blured_image

In [32]:
def AddGaussianNoise(img):
    mean = 0 
    sigma = np.random.uniform(5 , 15) 
    noise = np.random.normal(mean , sigma , img.shape)
    noisy_image = img + noise 
    noisy_image = np.clip(noisy_image , 0 , 255).astype(np.uint8)
    return noisy_image

In [33]:
def saveImage(img , path , num):
    if img is None or img.size == 0:
        print(f"Warning: image {num} is empty. Skipping save.")
        return
    
    img_resized = cv2.resize(img, (256, 256))
    img_resize = cv2.cvtColor(img_resized , cv2.COLOR_BGR2RGB)
    folder_name = os.path.basename(path)
    cv2.imwrite(f"{path}\\{folder_name}_augImage_{num}.jpg", img_resized)
    print(f"augmented_image {num} saved successfully!")

In [35]:
def ApplyAugumentationTechniques(images , folderPath):
    n = 9 
    cnt = 0
    H, W, C = 256, 256, 3  
    augmented_image = np.zeros((H, W, C))
    for img in images:
      for i in range(n) :
        match i : 
          case 0 : 
            augmented_image = RotateImage(img)  
          case 1 :
            augmented_image = FlipImageHorizontally(img)
          case 2 :
            augmented_image = ChangeBrightness(img)
          case 3:
            augmented_image = ChangeContrast(img)
          case 4 :
            augmented_image = ZoomImage(img)
          case 5 : 
            augmented_image = FlipImageVertically(img)
          case 6:
            augmented_image = ChangeSaturation(img)
          case 7: 
            augmented_image = GaussianBlur(img)
          case 8:
            augmented_image = AddGaussianNoise(img)
        saveImage(augmented_image , folderPath , cnt) 
        cnt = cnt + 1
        
       
        

In [36]:
folderPaths = [
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\cardboard",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\glass",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\metal",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\paper",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\plastic",
    r"D:\Fourth_level\Semester_1\Machine learning\dataset\trash"  
      ] 
dataset_path = r"D:\Fourth_level\Semester_1\Machine learning\test_data"
SplitDataBeforeAugmentation(folderPaths , dataset_path)
for folder in folderPaths:
    images = []
    for filename in os.listdir(folder):
        file_path = os.path.join(folder,filename)
        if os.path.isfile(file_path) and filename.lower().endswith('.jpg'):
          img = cv2.imread(file_path)
          if img is None:
            os.remove(file_path)
            print(f"{file_path} is removed , The image is corrupted and won't open ")
            continue 
          images.append(img)
    ApplyAugumentationTechniques(images , folder)
    
            


Moved 82 images from cardboard to test set
Moved 109 images from glass to test set
Moved 95 images from metal to test set
Moved 122 images from paper to test set
Moved 105 images from plastic to test set
Moved 54 images from trash to test set

Total test images: 567
augmented_image 0 saved successfully!
augmented_image 1 saved successfully!
augmented_image 2 saved successfully!
augmented_image 3 saved successfully!
augmented_image 4 saved successfully!
augmented_image 5 saved successfully!
augmented_image 6 saved successfully!
augmented_image 7 saved successfully!
augmented_image 8 saved successfully!
augmented_image 9 saved successfully!
augmented_image 10 saved successfully!
augmented_image 11 saved successfully!
augmented_image 12 saved successfully!
augmented_image 13 saved successfully!
augmented_image 14 saved successfully!
augmented_image 15 saved successfully!
augmented_image 16 saved successfully!
augmented_image 17 saved successfully!
augmented_image 18 saved successfully!
au